In [40]:
import pandas as pd
import pprint
import spacy
from spacy import displacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk import ngrams
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

compT, compR, compG = [], [],[]
negT, negR, negG = [], [],[]
posT, posR, posG = [], [],[]
neuT, neuR, neuG = [], [],[]
SIAT,SIAR, SIAG = 0, 0, 0

uword,bword,tword, ufreq, bfreq, tfreq= [], [], [], [], [], []

In [41]:
#pip install pprint

In [42]:
#Crear una funcion para guardar puntajes de sentimiento con vaderSentiment
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment
def setSIA(field,pos,neg,neu,comp):
    for i in range(0, len(field)):
      SIA = getSIA(field[i])
      comp.append(SIA['compound'])
      neg.append(SIA['neg'])
      neu.append(SIA['neu'])
      pos.append(SIA['pos'])

In [43]:
def word_frequency(sentence):
    sentence =" ".join(sentence)
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    counted = Counter(new_tokens)
    counted_2= Counter(ngrams(new_tokens,2))
    counted_3= Counter(ngrams(new_tokens,3))
    word_freq = pd.DataFrame(counted.items(),columns=['word','frequency']).sort_values(by='frequency',ascending=False)
    word_pairs =pd.DataFrame(counted_2.items(),columns=['pairs','frequency']).sort_values(by='frequency',ascending=False)
    trigrams =pd.DataFrame(counted_3.items(),columns=['trigrams','frequency']).sort_values(by='frequency',ascending=False)
    return word_freq,word_pairs,trigrams

In [44]:
def miJuntarColumnas(df_final, textoRet,df_col):
    for i in range(len(df_final.index)):
        text = '-'.join(df_col.iloc[i])
        textoRet.append(text)
    return textoRet

In [45]:
#uword,ufreq,bword,bfreq,tword,tfreq
def miNgramSetter(df_final,df_col):
    for i in range(len(df_final.index)):
        textTweets = "".join(df_col.iloc[i])
        wordsT = word_tokenize(textTweets)
        unigramT,bigramT,trigramT = word_frequency(wordsT)
        uword.append(unigramT['word'].iloc[0])
        ufreq.append(unigramT['frequency'].iloc[0])
        #bigramas
        bword.append(bigramT['pairs'].iloc[0])
        bfreq.append(bigramT['frequency'].iloc[0])
        #trigramas
        tword.append(trigramT['trigrams'].iloc[0])
        tfreq.append(trigramT['frequency'].iloc[0])
        #df_final['unigram'].iloc[i] = unigramT
    return uword,ufreq,bword,bfreq,tword,tfreq

In [46]:
def miNERAnalyzer(df_con,df_col,df_NER):
    for i in range(len(df_con.index)):
        entities, labels = [],[]
        textTweets=""
        textTweets = "".join(df_col.iloc[i])
        doc = nlp(textTweets)
        for ent in doc.ents:
            entities.append(ent)
            labels.append(ent.label_)
        columnaEnNombre = 'Entidades'+str(i)
        columnaEtNombre = 'Etiqueta'+str(i)
        df = pd.DataFrame({columnaEnNombre:entities, columnaEtNombre:labels})
        df_NER = pd.concat([df_NER, df], axis = 1)

In [47]:
nlp = spacy.load('en_core_web_sm')

C:\Users\christian\miniconda3\lib\site-packages\spacy\util.py:730: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [52]:
#Traigo Datos
df_tweets = pd.read_csv('exportTweetData.csv')
df_news = pd.read_csv('exportNewsData.csv')
df_market = pd.read_csv('exportMarketData.csv')

#Combino para juntar todos los twitts en una fila por minuto
dataSetCombinadoTwitter = df_tweets.merge(df_market, how='inner', on='Date')
dataSetCombinadoGoogle = df_news.merge(df_market,how='inner', on='Date')

In [53]:
#agrupo todos los tweets por fecha
groupTwitter = dataSetCombinadoTwitter.groupby('Date')['Tweets'].unique()

df_finalT =  dataSetCombinadoTwitter.merge(groupTwitter, how='inner', on='Date')

#Limpio el DataFrame de repetidos e info no utilizada
tweets = df_finalT['Tweets_y']
textoTweets = []

df_finalT = df_finalT.drop_duplicates(subset=['Date'], keep='last')
df_finalT = df_finalT.drop(columns=['Tweets_y', 'Tweets_x','Subjectivity','Polarity','Volume'])
df_finalT['Tweets']=tweets
df_finalT = df_finalT.reset_index(drop=True)

textoTweets = miJuntarColumnas(df_finalT,textoTweets,df_finalT['Tweets'])
df_finalT['Tweets']=textoTweets

In [56]:
df_finalT.head(10)

,Date,Label,Open,Close,High,Low,Timestamp,Tweets
0,2021-06-27 23:59,1,30400.0,30421.23654981,30495.04219868,30383.0,1611791940000,Best riskadjusted returns in the past hours:...
1,2021-06-27 23:58,1,30443.0,30401.0,30443.0,30400.0,1611791880000,Wake me up when this thing breaks out don’t t...
2,2021-06-27 23:57,0,30447.0,30457.0,30461.0,30417.0,1611791820000,Believer Hopefully you set aside capital gain...
3,2021-06-27 23:56,1,30333.0,30445.0,30458.0,30333.0,1611791760000,"Watch how quickly the news narratives change, ..."
4,2021-06-27 23:55,1,30462.0,30350.0,30463.0,30350.0,1611791700000,"Unusual spike in $COMP/USDT , USD worth of $C..."
5,2021-06-27 23:54,1,30339.0,30461.0,30461.0,30339.0,1611791640000,"I just entered to win a pool of , Click the ..."
6,2021-06-27 23:53,1,30315.953652,30338.994468,30348.0,30310.193448,1611791580000,is this what denial feels like? $BTC-Bitcoin w...
7,2021-06-27 23:52,1,30349.0,30351.0,30368.0,30330.0,1611791520000,$BTC BUYING PRESSURE ALERT Price trading aro...
8,2021-06-27 23:51,1,30358.0,30349.0,30360.115216,30316.0,1611791460000,Hi guys can anyone help me and lend me some us...
9,2021-06-27 23:50,1,30294.0,30361.07525,30374.0,30282.352462,1611791400000,$BTC Standard Deviation levels PDF available f...


In [54]:
#agrupo todos las noticias por fecha
groupGoogle = dataSetCombinadoGoogle.groupby('Date')['Desc'].unique()
df_finalG =  dataSetCombinadoGoogle.merge(groupGoogle, how='inner', on='Date')
#Limpio el DataFrame de repetidos e info no utilizada
news = df_finalG['Desc_y']
textoNews = []

df_finalG = df_finalG.drop_duplicates(subset=['Date'], keep='last')
df_finalG = df_finalG.drop(columns=['Desc_y', 'Desc_x','Volume','Title','Media'])
df_finalG['Desc']=news
df_finalG = df_finalG.reset_index(drop=True)

textoNews = miJuntarColumnas(df_finalG,textoNews,df_finalG['Desc'])
df_finalG['Desc']=textoNews

In [55]:
df_finalG.head(10)

,Unnamed: 0,Date,Open,Close,High,Low,Timestamp,Desc
0,9,2021-06-27 23:44,30188.0,30171.0,30226.0,30153.0,1611791040000,Bitcoin will not just be a companion currency ...


In [57]:
setSIA(df_finalT['Tweets'],posT,negT,neuT,compT)
df_finalT['Compound'] = compT
df_finalT['Negative'] = negT
df_finalT['Neutral'] = neuT
df_finalT['Positive'] = posT

In [58]:
setSIA(df_finalG['Desc'],posG,negG,neuG,compG)
df_finalG['Compound'] = compG
df_finalG['Negative'] = negG
df_finalG['Neutral'] = neuG
df_finalG['Positive'] = posG

In [59]:
#Analisis de Word Frequency y ngramas
#Junto todo el texto de cada fila y una vez hecho eso lo agrego al dataframe

uword,ufreq,bword,bfreq,tword,tfreq =  miNgramSetter(df_finalT,df_finalT['Tweets'])
    
formatoDataSetNgramas = {'Unigrama': uword,'FreqU': ufreq, 'Bigrama':bword, 'FreqB':bfreq, 'Trigrama':tword, 'FreqT':tfreq}
df_ngramas = pd.DataFrame(data=formatoDataSetNgramas)
df_concatenadoT = pd.concat([df_finalT, df_ngramas], axis=1)

In [60]:
#df_concatenadoT.head(10)

In [61]:
#Tengo que volver a vaciar estas listas
uword,bword,tword, ufreq, bfreq, tfreq= [], [], [], [], [], []

In [62]:
uword,ufreq,bword,bfreq,tword,tfreq =  miNgramSetter(df_finalG,df_finalG['Desc'])
    
formatoDataSetNgramas = {'Unigrama': uword,'FreqU': ufreq, 'Bigrama':bword, 'FreqB':bfreq, 'Trigrama':tword, 'FreqT':tfreq}
df_ngramas = pd.DataFrame(data=formatoDataSetNgramas)
df_concatenadoG = pd.concat([df_finalG, df_ngramas], axis=1)

In [63]:
#df_concatenadoG.head(10)

In [38]:
#Analisis de NER. Me voy a quedar con la entidad mas repetida y su label
allEnt, entFin, allLabls, lablsFin = [],[],[],[]
df_NER = pd.DataFrame()
#miNERAnalyzer(df_concatenadoT, df_finalT['Tweets'],df_NER)
for i in range(len(df_concatenadoT.index)):
    entities, labels = [],[]
    textTweets=""
    textTweets = "".join(df_finalT['Tweets'].iloc[i])
    doc = nlp(textTweets)
    for ent in doc.ents:
        entities.append(ent)
        labels.append(ent.label_)
    columnaEnNombre = 'Entidades'+str(i)
    columnaEtNombre = 'Etiqueta'+str(i)
    df = pd.DataFrame({columnaEnNombre:entities, columnaEtNombre:labels})
    df_NER = pd.concat([df_NER, df], axis = 1)
df_NER = df_NER.dropna()
   

In [39]:
df_NER.head(10)  

,Entidades0,Etiqueta0,Entidades1,Etiqueta1,Entidades2,Etiqueta2,Entidades3,Etiqueta3,Entidades4,Etiqueta4,...,Entidades53,Etiqueta53,Entidades54,Etiqueta54,Entidades55,Etiqueta55,Entidades56,Etiqueta56,Entidades57,Etiqueta57
0,"(the, past, , hours)",TIME,"($, QNT, $)",MONEY,(BTC),ORG,"($, xrp, $)",MONEY,"(COMP, /, USDT)",ORG,...,(tomorrow),DATE,(ETH),MONEY,(Matic),NORP,"(’m, not, Hodling, $, BTC)",ORG,"(ONNext, day)",DATE
1,(ICP),MONEY,(BTC),ORG,(BTC),ORG,(BTC),ORG,(COMP),ORG,...,"(days, %)",PERCENT,(BTC),ORG,(BTC),ORG,"($, BTC, $, ETH, $)",MONEY,(first),ORDINAL
2,(ETH),MONEY,"(XRP, $, ALBT, $)",ORG,(DOGE),MONEY,"($, BTC, $, ETH, $)",MONEY,(COMP),MONEY,...,(BTC),ORG,(Crypto),ORG,"($, ETH, $)",MONEY,(BNB),ORG,(-A),ORG
3,($),CARDINAL,(DAG),ORG,(BTCDEC),ORG,(ADA),ORG,"($, USDT$FTX, $)",MONEY,...,(Gutsy),PERSON,(Bitcoin),PERSON,"(MATIC, -, )",PERSON,(BTC),ORG,(BTC),MONEY
4,(BTC),ORG,(ZCX),ORG,(BTC),ORG,($),CARDINAL,"($, BTC, $, ETH, $)",MONEY,...,"(K, $, BTC)",ORG,"($, xrp, $)",MONEY,"(Antminer, Ss)",PERSON,"(Price, Drop)",PERSON,"(this, morning)",TIME
5,"($, ETH, $)",MONEY,"($, VET, $, DOGE, $, XLM, $)",MONEY,(Smart),ORG,"(CNBC, %)",ORG,(BTC),ORG,...,(WEEKEND),ORG,"($, link, $, UNI, $)",MONEY,"(Th, /, kW)",ORG,"(Last, Year)",DATE,(BTC),PRODUCT


In [66]:
df_NERG = pd.DataFrame()
#miNERAnalyzer(df_concatenadoT, df_finalT['Tweets'],df_NER)
for i in range(len(df_concatenadoG.index)):
    entities, labels = [],[]
    textTweets=""
    textTweets = "".join(df_finalG['Desc'].iloc[i])
    doc = nlp(textTweets)
    for ent in doc.ents:
        entities.append(ent)
        labels.append(ent.label_)
    columnaEnNombre = 'Entidades'+str(i)
    columnaEtNombre = 'Etiqueta'+str(i)
    df = pd.DataFrame({columnaEnNombre:entities, columnaEtNombre:labels})
    df_NERG = pd.concat([df_NERG, df], axis = 1)
df_NERG = df_NERG.dropna()

In [67]:
df_NERG.head(10)  

,Entidades0,Etiqueta0
0,(U.S.),GPE
1,(Bukele),ORG
2,(Bitcoin),PERSON
3,"(Central, and, South, America)",LOC
4,"(El, Salvador)",GPE
5,"(Nayib, Bukele)",PERSON
6,"(Wood, 's, Ark, Investment, Management)",ORG
7,(Bitcoin),PERSON
8,(ARK),ORG
9,(Europe),LOC


In [68]:
#Analizo el sentimiento total de todos sumando los compund que es de por si la media normalizada entre el positivo y negativo
sumatoriaCompoundTwitter = df_concatenadoT['Compound'].sum() / len(df_concatenadoT.index)
#Si es mayor a 0,5 indica un sentimiento positivo
if sumatoriaCompoundTwitter > 0.5:
    print("El sentimiento es positivo y es de", sumatoriaCompoundTwitter)
elif sumatoriaCompoundTwitter == 0.5:
    print("El sentimiento es neutro", sumatoriaCompoundTwitter)
else:
    print("El sentimiento es negativo", sumatoriaCompoundTwitter)

El sentimiento es positivo y es de 0.8031758620689655


In [69]:
#Analizo el sentimiento total de todos sumando los compund que es de por si la media normalizada entre el positivo y negativo
sumatoriaCompoundGoogle = df_concatenadoG['Compound'].sum() / len(df_concatenadoG.index)
#Si es mayor a 0,5 indica un sentimiento positivo
if sumatoriaCompoundGoogle > 0.5:
    print("El sentimiento es positivo y es de", sumatoriaCompoundGoogle)
elif sumatoriaCompoundGoogle == 0.5:
    print("El sentimiento es neutro", sumatoriaCompoundGoogle)
else:
    print("El sentimiento es negativo", sumatoriaCompoundGoogle)

El sentimiento es positivo y es de 0.9231


In [72]:
print("El sentimiento promedio de las dos fuentes para el dia 27/6/2021 es de", (sumatoriaCompoundGoogle+sumatoriaCompoundTwitter)/2)

El sentimiento promedio de las dos fuentes para el dia 27/6/2021 es de 0.8631379310344828
